In [1]:
import boto3
import sagemaker

from sagemaker import get_execution_role

region = 'ca-central-1'
role_name = 'sagemaker-fullaccess'
bucket = 'luntai-sagemaker-learning'

role = boto3.client('iam').get_role(RoleName='sagemaker-fullaccess')['Role']['Arn']
role

'arn:aws:iam::981401473042:role/sagemaker-fullaccess'

In [2]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.pytorch import PyTorchModel

In [10]:
model = HuggingFaceModel(
    py_version = "py310",
    entry_point = "inference_code.py",
    transformers_version  = "4.28", # transformer version
    pytorch_version = "2.0.0",
    model_data = f"s3://{bucket}/model.tar.gz",
    role = role,
)

In [13]:
model = PyTorchModel(
    py_version = "py310",
    entry_point = "inference_code.py",
    framework_version  = "2.0.0", # pytorch version
    model_data = f"s3://{bucket}/model.tar.gz",
    role = role,
)

In [ ]:
predictor = model.deploy(
    instance_type='ml.m5.xlarge', 
    initial_instance_count=1,
    endpoint_name='text-summarizer-wcd'
)

In [9]:
import json

sm = sagemaker.Session().sagemaker_runtime_client

prompt = {
  "articles": ["(CNN)Michele Bachmann is comparing President Obama to the co-pilot of the doomed Germanwings flight. \"With his Iran deal, Barack Obama is for the 300 million souls of the United States what Andreas Lubitz was for the 150 souls on the German Wings flight - a deranged pilot flying his entire nation into the rocks,\" the Minnesota Republican and former representative wrote in a Facebook comment posted March 31. \"After the fact, among the smoldering remains of American cities, the shocked survivors will ask, why did he do it?\" Andreas Lubitz, the co-pilot of Germanwings Flight 9525, is accused by authorities of deliberately crashing the plane in the French Alps. He died in the crash along with 149 other crew and passengers. The motive of the March 24 crash is under investigation, though investigators are looking in to whether Lubitz feared a medical condition would cause him to lose his pilot's license. Many comments posted on her Facebook page blasted the former representative. Melissa Coca wrote, \"Comparing this tragedy to anything is moronic and despicable.\" Michael J Pristash wrote, \"Your allusion is so inappropriate and divisive, not to mention disrespectful on so many levels. Shame on you.\" Some also accused her of taking desperate measures to stay in the public eye. Lynda Anderson wrote, \"Posting outrageous things in a pathetic attempt to stay relevant?\" Negotiations are coming down to the wire between Iran, the United States and other nations on restricting Tehran's nuclear program to prevent the ability to develop an atomic bomb. One deadline passed Tuesday, but there is a June 30 deadline for a comprehensive deal -- with all technical and diplomatic impasses fully worked out. Bachmann is no stranger to voicing her opinion on the President's dealing with Iran, personally telling him to \"bomb Iran\" during the 2014 White House Christmas Party. \"I turned to the president and I said, something to the effect of, 'Mr. President, you need to bomb the Iranian nuclear facilities, because if you don't, Iran will have a nuclear weapon on your watch and the course of world history will change,'\" she told the Washington Free Beacon. The congresswoman, who sought the GOP presidential nomination in 2012, said Obama had a \"condescending smile on his face and laughed at me.\" She said he told her: \"Well Michele, it's just not that easy."]
}

response = sm.invoke_endpoint(
    EndpointName="huggingface-pytorch-inference-2023-06-16-14-46-41-544", 
    Body=json.dumps(prompt), 
    ContentType="application/json"
)

response["Body"].read()

b'[\n  "Michele Bachmann is comparing President Obama to the co-pilot of the doomed Germanwings flight."\n]'

In [13]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-06-16-14-46-41-544'

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=f"s3://{bucket}/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

In [ ]:
# example request: you always need to define "inputs"
data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

# request
predictor.predict(data)

In [4]:
model = HuggingFaceModel(
    py_version = "py39",
    entry_point = "inference_code.py",
    transformers_version  = "4.26", # transformer version
    pytorch_version = "1.13",
    model_data = f"s3://{bucket}/text-summarizer/torch/model.tar.gz",
    role = role,
)
# deploy model to SageMaker Inference
predictor = model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

----!

In [12]:
data = {
	"articles": ["One of the key ideas behind why you use instruction style prompts is that you can steer the model towards what you define as your goal state output, vs what the model, a user, or a different engineer believes it is. For our use case this just means we can use the prompt to help the model better understand what information to include in our summary, what format we want it in, extractive vs abstractive etc. If we were to provide a prompt such as “summary this document” our model doesn’t have much information around what would be a good output for us, and will rely on its idea of what a good output is. The flip to this would be if we provide a prompt such as “Extract exact key information from the provided text in 5 sentences that focuses on xyz”. You can see how we’ve provided a much deeper idea to the model of what a good output is for us (goal state output).GPT-4 is much better than GPT-3 at understanding more complex prompt instructions in zero-shot scenarios, or scenarios where we don’t provide exact input and output examples of the summaries of example documents. We’ll see in examples below that the prompts we can provide can be much more complex, contain multiple steps, have clear guidelines, and much more that GPT-3 simply struggles with. The model is also very good at understanding the idea of pretending to be certain roles, such as a creative writer, an engineer, a student etc. This role play idea is something that we initially weren’t too keen on, as the value add didn’t make much sense over better prompts or prompt examples. Through testing and even integrating it into production we’ve found this role play idea very useful for less experienced prompt engineers or giving the model a more generalized idea of what output goal state we want to steer towards."],
}
predictor.predict(data)

['We’ll see in examples below that the prompts we can provide can be much more complex, contain multiple steps, have clear guidelines, and much more that GPT-3 simply struggles with.']